In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import networkx as nx
from sklearn.neighbors import NearestNeighbors

In [2]:
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['Class'] = data.target

features = data.feature_names.tolist()
X = df[features].values
y = df['Class'].values

X_train_raw, X_test_raw, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, np.arange(len(df)),
    test_size=0.30,
    stratify=y,
    random_state=42
)

scaler = StandardScaler().fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_test  = scaler.transform(X_test_raw)

In [3]:


def run_aco_with_knn(
    X_train, y_train, X_test,
    k=5, n_ants=50, n_iter=30, rho=0.1, rng_seed=42,
    alpha_edge=1.0, beta_phero=1.0, tau=1.0,
    max_steps=100, eps=1e-12
):
    """
    Compute Pheromone-Score (PS) via kNN graph + ACO on TRAIN only (no leakage),
    then estimate PS for TEST by neighbor-weighted averaging.

    Assumptions:
      - X_train / X_test are already standardized (e.g., StandardScaler fit on train).

    Parameters:
      k (int): number of neighbors for kNN graph (on TRAIN).
      n_ants (int): ants per iteration.
      n_iter (int): number of ACO iterations.
      rho (float): pheromone evaporation rate in (0,1).
      rng_seed (int): random seed for reproducibility.
      alpha_edge (float): exponent on edge weight (similarity emphasis).
      beta_phero (float): exponent on (1 + pheromone) (exploitation emphasis).
      tau (float): softmax temperature (lower → sharper selection).
      max_steps (int): maximum steps per ant walk (prevents overly long paths).
      eps (float): numerical stability constant.

    Returns:
      pheromone_train_scaled (np.ndarray): PS in [0,1] for TRAIN nodes (shape: [n_train]).
      pheromone_test_scaled  (np.ndarray): PS estimate in [0,1] for TEST nodes  (shape: [n_test]).
    """
    # kNN on TRAIN
    nbrs = NearestNeighbors(n_neighbors=k, metric='euclidean').fit(X_train)
    dist_train, ind_train = nbrs.kneighbors(X_train)

    G = nx.Graph()
    n_tr = len(X_train)
    for i in range(n_tr):
        G.add_node(i, label=int(y_train[i]), pheromone=0.0)

    # build weighted edges using returned distances (avoid recomputing norms)
    for i, neigh in enumerate(ind_train):
        for jj, j in enumerate(neigh):
            if i == j:
                continue
            d_ij = dist_train[i, jj]
            w = 1.0 / (1.0 + d_ij)
            G.add_edge(i, j, weight=w)

    majority_nodes = [i for i in range(n_tr) if y_train[i] == 1]
    minority_nodes = set([i for i in range(n_tr) if y_train[i] == 0])

    rng = np.random.default_rng(rng_seed)

    # ACO loop
    for _ in range(n_iter):
        for _ in range(n_ants):
            current = rng.choice(majority_nodes)
            visited = [current]
            steps = 0

            while steps < max_steps:
                neighs = [n for n in G.neighbors(current) if n not in visited]
                if not neighs:
                    break

                # move probability ∝ (edge_weight^alpha_edge) * ((1+pheromone)^beta_phero)
                raw = []
                for n in neighs:
                    e = G[current][n]['weight'] ** alpha_edge
                    p = (1.0 + G.nodes[n]['pheromone']) ** beta_phero
                    raw.append(e * p)
                raw = np.array(raw, dtype=float)

                # temperature-scaled softmax for stability/controllability
                logits = raw / max(tau, eps)
                logits -= logits.max()  # numerical stability
                probs = np.exp(logits)
                s = probs.sum()
                if s <= eps:
                    break
                probs /= s

                next_node = rng.choice(neighs, p=probs)
                visited.append(next_node)
                current = next_node
                steps += 1

                if current in minority_nodes:
                    for idx in visited:
                        G.nodes[idx]['pheromone'] += 1.0
                    break

        # evaporation
        for n in G.nodes:
            G.nodes[n]['pheromone'] *= (1.0 - rho)

    # normalize PS on TRAIN
    pheromone_train = np.array([G.nodes[i]['pheromone'] for i in range(n_tr)], dtype=float)
    scaler = MinMaxScaler().fit(pheromone_train.reshape(-1, 1))
    pheromone_train_scaled = scaler.transform(pheromone_train.reshape(-1, 1)).ravel()

    # estimate PS for TEST via inverse-distance weighted average of TRAIN neighbors
    dist_te, ind_te = nbrs.kneighbors(X_test, n_neighbors=k)
    w_te = 1.0 / (eps + dist_te)
    w_te = w_te / (w_te.sum(axis=1, keepdims=True) + eps)
    pheromone_test_scaled = (w_te * pheromone_train_scaled[ind_te]).sum(axis=1)

    return pheromone_train_scaled, pheromone_test_scaled


In [4]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
import time
from itertools import product
from joblib import Parallel, delayed

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    f1_score, precision_score, recall_score, accuracy_score,
    average_precision_score, roc_auc_score
)

# ----------------------------
# Assumption: X, y and the function run_aco_with_knn are already defined
# y: 0 = minority, 1 = majority
# ----------------------------


# ======= Parameter space =======
# ACO/kNN
k_values            = [5, 10]
n_ants_values       = [50, 100]
n_iter_values       = [20, 30]
rho_values          = [0.1, 0.3]

# ACO movement policy
alpha_edge_values   = [0.5, 1.0, 2.0]
beta_phero_values   = [0.5, 1.0, 2.0]
tau_values          = [0.3, 1.0, 3.0]

# Training sample weighting
alpha_values        = [1.0]
beta_values         = [1.0, 2.0]
gamma_values        = [1.0]
delta_values        = [0.0, 0.5]

# K-Fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# ----------------------------
# Prepare fold data (scaling applied on train only)
# ----------------------------
fold_data = []  # [(fold, X_tr, y_tr, X_te, y_te)]
for f, (tr_idx, te_idx) in enumerate(cv.split(X, y)):
    X_tr_raw, X_te_raw = X[tr_idx], X[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]
    scaler = StandardScaler().fit(X_tr_raw)
    X_tr = scaler.transform(X_tr_raw)
    X_te = scaler.transform(X_te_raw)
    fold_data.append((f, X_tr, y_tr, X_te, y_te))

# ----------------------------
# 1) PRECOMPUTE PS with parallelization
# ----------------------------
aco_grid = list(product(
    k_values, n_ants_values, n_iter_values, rho_values,
    alpha_edge_values, beta_phero_values, tau_values
))

def compute_ps_for_combo(combo):
    # Each combo is evaluated across all folds
    k, n_ants, n_iter, rho, alpha_edge, beta_phero, tau = combo
    outs = []
    for (f, X_tr, y_tr, X_te, y_te) in fold_data:
        # Shorten ant paths → faster execution
        p_tr, p_te = run_aco_with_knn(
            X_tr, y_tr, X_te,
            k=k, n_ants=n_ants, n_iter=n_iter, rho=rho, rng_seed=42,
            alpha_edge=alpha_edge, beta_phero=beta_phero, tau=tau,
            max_steps=15*k
        )
        outs.append((f, p_tr, p_te))
    return combo, outs

n_jobs = max(2, min(8, (os.cpu_count() or 4)//2))
t0 = time.perf_counter()
results = Parallel(n_jobs=n_jobs, prefer="processes", verbose=10)(
    delayed(compute_ps_for_combo)(combo) for combo in aco_grid
)
t1 = time.perf_counter()

# Build cache
ps_cache = {}
for combo, outs in results:
    k, n_ants, n_iter, rho, alpha_edge, beta_phero, tau = combo
    for (f, p_tr, p_te) in outs:
        ps_cache[(f, k, n_ants, n_iter, rho, alpha_edge, beta_phero, tau)] = (p_tr, p_te)

# ----------------------------
# 2) Evaluate weightings using precomputed PS
# ----------------------------
def eval_with_cached_ps(k, n_ants, n_iter, rho,
                        alpha_edge, beta_phero, tau,
                        alpha, beta, gamma, delta,
                        rf_n_estimators=100):
    f1_m_list, prec_m_list, rec_m_list = [], [], []
    macro_f1_list, acc_list = [], []
    aucpr_m_list, aucroc_m_list = [], []
    fit_time_list, pred_time_list = [], []

    for (f, X_tr, y_tr, X_te, y_te) in fold_data:
        phero_tr, phero_te = ps_cache[(f, k, n_ants, n_iter, rho, alpha_edge, beta_phero, tau)]
        X_tr_phero = np.column_stack([X_tr, phero_tr])
        X_te_phero = np.column_stack([X_te, phero_te])

        # Sample weighting
        w_tr = np.where(
            y_tr == 1,
            gamma + delta * phero_tr,
            alpha + beta  * phero_tr
        )

        clf = RandomForestClassifier(
            n_estimators=rf_n_estimators,
            random_state=42,
            n_jobs=1
        )

        t_fit0 = time.perf_counter()
        clf.fit(X_tr_phero, y_tr, sample_weight=w_tr)
        t_fit1 = time.perf_counter()

        t_pred0 = time.perf_counter()
        proba1 = clf.predict_proba(X_te_phero)[:, 1]
        y_pred = (proba1 >= 0.5).astype(int)
        t_pred1 = time.perf_counter()

        # Metrics for minority class = 0
        f1_m   = f1_score(y_te, y_pred, pos_label=0)
        prec_m = precision_score(y_te, y_pred, pos_label=0, zero_division=0)
        rec_m  = recall_score(y_te, y_pred, pos_label=0)
        macroF1 = f1_score(y_te, y_pred, average='macro')
        acc    = accuracy_score(y_te, y_pred)

        proba_min = 1.0 - proba1
        aucpr_m   = average_precision_score(1 - y_te, proba_min)
        aucroc_m  = roc_auc_score(1 - y_te, proba_min)

        f1_m_list.append(f1_m)
        prec_m_list.append(prec_m)
        rec_m_list.append(rec_m)
        macro_f1_list.append(macroF1)
        acc_list.append(acc)
        aucpr_m_list.append(aucpr_m)
        aucroc_m_list.append(aucroc_m)
        fit_time_list.append(t_fit1 - t_fit0)
        pred_time_list.append(t_pred1 - t_pred0)

    return {
        "k": k, "n_ants": n_ants, "n_iter": n_iter, "rho": rho,
        "alpha_edge": alpha_edge, "beta_phero": beta_phero, "tau": tau,
        "alpha": alpha, "beta": beta, "gamma": gamma, "delta": delta,
        "F1_minority_mean":   float(np.mean(f1_m_list)),
        "F1_minority_std":    float(np.std(f1_m_list)),
        "Precision_min_mean": float(np.mean(prec_m_list)),
        "Recall_min_mean":    float(np.mean(rec_m_list)),
        "MacroF1_mean":       float(np.mean(macro_f1_list)),
        "Accuracy_mean":      float(np.mean(acc_list)),
        "AUC_PR_min_mean":    float(np.mean(aucpr_m_list)),
        "AUC_PR_min_std":     float(np.std(aucpr_m_list)),
        "AUC_ROC_min_mean":   float(np.mean(aucroc_m_list)),
        "AUC_ROC_min_std":    float(np.std(aucroc_m_list)),
        "TrainTime_s_mean":   float(np.mean(fit_time_list)),
        "PredTime_s_mean":    float(np.mean(pred_time_list)),
    }

rows = []
start_eval = time.perf_counter()
for (k, n_ants, n_iter, rho, alpha_edge, beta_phero, tau) in aco_grid:
    for (alpha, beta, gamma, delta) in product(alpha_values, beta_values, gamma_values, delta_values):
        rows.append(
            eval_with_cached_ps(k, n_ants, n_iter, rho,
                                alpha_edge, beta_phero, tau,
                                alpha, beta, gamma, delta)
        )
end_eval = time.perf_counter()

results_df = pd.DataFrame(rows)

# ----------------------------
# Final report
# ----------------------------
print(f"[PS Precompute] combos: {len(aco_grid)}, n_jobs={n_jobs}, time: {t1 - t0:.2f}s")
print(f"[RF Eval] combos: {len(rows)}, time: {end_eval - start_eval:.2f}s")

topN = 15
show_cols = [
    "k","n_ants","n_iter","rho","alpha_edge","beta_phero","tau",
    "alpha","beta","gamma","delta",
    "F1_minority_mean","F1_minority_std",
    "AUC_PR_min_mean","AUC_PR_min_std",
    "Precision_min_mean","Recall_min_mean",
    "MacroF1_mean","Accuracy_mean",
    "TrainTime_s_mean","PredTime_s_mean"
]

print("\n=== Top by F1_minority_mean ===")
print(results_df.sort_values("F1_minority_mean", ascending=False)[show_cols].head(topN))

print("\n=== Top by AUC_PR_min_mean ===")
print(results_df.sort_values("AUC_PR_min_mean", ascending=False)[show_cols].head(topN))

# results_df.to_csv("ps_weighted_full_grid_results.csv", index=False)


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    3.6s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    5.6s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    8.1s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   11.1s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   13.9s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:   17.6s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:   22.5s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:   29.6s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:   35.5s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:   42.7s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   52.7s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:  1.5min
[Parallel(

[PS Precompute] combos: 432, n_jobs=6, time: 447.78s
[RF Eval] combos: 1728, time: 537.35s

=== Top by F1_minority_mean ===
     k  n_ants  n_iter  rho  alpha_edge  beta_phero  tau  alpha  beta  gamma  \
513  5     100      20  0.1         2.0         0.5  3.0    1.0   1.0    1.0   
93   5      50      20  0.1         2.0         1.0  3.0    1.0   1.0    1.0   
418  5      50      30  0.3         2.0         1.0  3.0    1.0   2.0    1.0   
515  5     100      20  0.1         2.0         0.5  3.0    1.0   2.0    1.0   
186  5      50      20  0.3         2.0         0.5  1.0    1.0   2.0    1.0   
239  5      50      30  0.1         0.5         1.0  3.0    1.0   2.0    1.0   
512  5     100      20  0.1         2.0         0.5  3.0    1.0   1.0    1.0   
731  5     100      30  0.1         2.0         0.5  3.0    1.0   2.0    1.0   
730  5     100      30  0.1         2.0         0.5  3.0    1.0   2.0    1.0   
729  5     100      30  0.1         2.0         0.5  3.0    1.0   1.0    1.0

In [6]:
# -*- coding: utf-8 -*-

import networkx as nx
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, average_precision_score
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier


# -----------------------------
# Data: Breast Cancer (0 = malignant, minority; 1 = benign, majority)
# -----------------------------
data = load_breast_cancer()
X = data.data
y = data.target  # 0 = minority (malignant), 1 = majority (benign)

# -----------------------------
# Function: ACO on kNN graph only on TRAIN (no leakage)
# Estimate pheromone for TEST from TRAIN neighbors
# Movement policy: softmax with temperature τ and weighting (edge^alpha_edge) * (1+pheromone)^beta_phero
# -----------------------------
def run_aco_with_knn(
    X_train, y_train, X_test,
    k=5, n_ants=100, n_iter=20, rho=0.1, rng_seed=42,
    alpha_edge=1.0, beta_phero=1.0, tau=1.0,
    max_steps=None, eps=1e-12
):
    # kNN based on TRAIN
    nbrs = NearestNeighbors(n_neighbors=k, metric='euclidean').fit(X_train)
    dist_train, ind_train = nbrs.kneighbors(X_train)

    # Weighted undirected graph
    G = nx.Graph()
    n_tr = X_train.shape[0]
    for i in range(n_tr):
        G.add_node(i, label=int(y_train[i]), pheromone=0.0)

    # Edges with weight 1/(1+d) (using kneighbors for efficiency)
    for i, neigh in enumerate(ind_train):
        for jj, j in enumerate(neigh):
            if i == j:
                continue
            d_ij = dist_train[i, jj]
            w = 1.0 / (1.0 + d_ij)
            G.add_edge(i, j, weight=w)

    majority_nodes = [i for i in range(n_tr) if y_train[i] == 1]
    minority_nodes = set([i for i in range(n_tr) if y_train[i] == 0])

    rng = np.random.default_rng(rng_seed)
    if max_steps is None:
        max_steps = 3 * k  # reasonable cap for speed

    # ACO iterations
    for _ in range(n_iter):
        for _ in range(n_ants):
            current = rng.choice(majority_nodes)
            visited = [current]
            steps = 0

            while steps < max_steps:
                neighs = [n for n in G.neighbors(current) if n not in visited]
                if not neighs:
                    break

                # Move probability ∝ (edge_weight^alpha_edge) * ((1+pheromone)^beta_phero)
                raw = []
                for n in neighs:
                    e = G[current][n]['weight'] ** alpha_edge
                    p = (1.0 + G.nodes[n]['pheromone']) ** beta_phero
                    raw.append(e * p)
                raw = np.asarray(raw, dtype=float)

                # Softmax with temperature τ
                logits = raw / max(tau, eps)
                logits -= logits.max()
                probs = np.exp(logits)
                s = probs.sum()
                if s <= eps:
                    break
                probs /= s

                next_node = rng.choice(neighs, p=probs)
                visited.append(next_node)
                current = next_node
                steps += 1

                # Reaching a minority node → reinforce the whole path
                if current in minority_nodes:
                    for idx in visited:
                        G.nodes[idx]['pheromone'] += 1.0
                    break

        # Pheromone evaporation
        for n in G.nodes:
            G.nodes[n]['pheromone'] *= (1.0 - rho)

    # Normalize PS on TRAIN
    phero_tr = np.array([G.nodes[i]['pheromone'] for i in range(n_tr)], dtype=float)
    phero_scaler = MinMaxScaler().fit(phero_tr.reshape(-1, 1))
    phero_tr_s = phero_scaler.transform(phero_tr.reshape(-1, 1)).ravel()

    # Estimate PS for TEST using inverse-distance weighted average from TRAIN
    dist_te, ind_te = nbrs.kneighbors(X_test, n_neighbors=k)
    w_te = 1.0 / (eps + dist_te)
    w_te = w_te / (w_te.sum(axis=1, keepdims=True) + eps)
    phero_te_s = (w_te * phero_tr_s[ind_te]).sum(axis=1)

    return phero_tr_s, phero_te_s


# -----------------------------
# Helper: Minority metrics (class 0)
# -----------------------------
def minority_metrics_from_proba(y_true, proba_class1):
    # For minority = 0, probability of minority = 1 - proba_class1
    proba_minority = 1.0 - proba_class1
    y_pred = (proba_class1 >= 0.5).astype(int)  # 1 = majority
    f1_min = f1_score(y_true, y_pred, pos_label=0)
    aucpr_min = average_precision_score(1 - y_true, proba_minority)
    return f1_min, aucpr_min


# -----------------------------
# Evaluate our method (two modes: PS feature only / PS feature + weighting)
# -----------------------------
def eval_my_method(
    X, y, cv,
    # ACO/kNN
    k=5, n_ants=50, n_iter=30, rho=0.3,
    alpha_edge=2.0, beta_phero=0.5, tau=3.0, max_steps=None,
    # Training weighting
    alpha=1.0, beta=2.0, gamma=1.0, delta=0.5,
    use_weights=True, random_state=42
):
    f1s, aucprs = [], []

    for tr_idx, te_idx in cv.split(X, y):
        X_tr_raw, X_te_raw = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]

        scaler = StandardScaler().fit(X_tr_raw)
        X_tr = scaler.transform(X_tr_raw)
        X_te = scaler.transform(X_te_raw)

        # PS computed only from TRAIN
        phero_tr, phero_te = run_aco_with_knn(
            X_tr, y_tr, X_te,
            k=k, n_ants=n_ants, n_iter=n_iter, rho=rho, rng_seed=random_state,
            alpha_edge=alpha_edge, beta_phero=beta_phero, tau=tau, max_steps=max_steps
        )

        X_tr_phero = np.column_stack([X_tr, phero_tr])
        X_te_phero = np.column_stack([X_te, phero_te])

        sample_weight = None
        if use_weights:
            sample_weight = np.where(
                y_tr == 1,
                gamma + delta * phero_tr,      # Majority: hard negatives get extra weight
                alpha + beta  * phero_tr       # Minority
            )

        clf = RandomForestClassifier(n_estimators=100, random_state=random_state)
        clf.fit(X_tr_phero, y_tr, sample_weight=sample_weight)
        proba1 = clf.predict_proba(X_te_phero)[:, 1]

        f1_min, aucpr_min = minority_metrics_from_proba(y_te, proba1)
        f1s.append(f1_min)
        aucprs.append(aucpr_min)

    return {
        "F1_minority_mean": np.mean(f1s), "F1_minority_std": np.std(f1s),
        "AUC_PR_minority_mean": np.mean(aucprs), "AUC_PR_minority_std": np.std(aucprs)
    }


# -----------------------------
# Baselines (strong and weak)
# -----------------------------
def eval_baselines(X, y, cv, random_state=42):
    rows = []

    for tr_idx, te_idx in cv.split(X, y):
        X_tr_raw, X_te_raw = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]

        scaler = StandardScaler().fit(X_tr_raw)
        X_tr = scaler.transform(X_tr_raw)
        X_te = scaler.transform(X_te_raw)

        # Dummy: always predict majority
        clf_dummy = DummyClassifier(strategy="most_frequent")
        clf_dummy.fit(X_tr, y_tr)
        proba1 = clf_dummy.predict_proba(X_te)[:, 1]
        rows.append(("Dummy_MostFrequent", *minority_metrics_from_proba(y_te, proba1)))

        # Plain RF
        rf_plain = RandomForestClassifier(n_estimators=100, random_state=random_state)
        rf_plain.fit(X_tr, y_tr)
        proba1 = rf_plain.predict_proba(X_te)[:, 1]
        rows.append(("RF_plain", *minority_metrics_from_proba(y_te, proba1)))

        # Logistic Regression without class_weight
        logreg = LogisticRegression(max_iter=1000, solver="lbfgs", random_state=random_state)
        logreg.fit(X_tr, y_tr)
        proba1 = logreg.predict_proba(X_te)[:, 1]
        rows.append(("LogReg_noCW", *minority_metrics_from_proba(y_te, proba1)))

        # Simple KNN
        knn = KNeighborsClassifier(n_neighbors=5, weights="uniform")
        knn.fit(X_tr, y_tr)
        proba1 = knn.predict_proba(X_te)[:, 1]
        rows.append(("KNN_k5_uniform", *minority_metrics_from_proba(y_te, proba1)))

        # Shallow Decision Tree
        dt = DecisionTreeClassifier(max_depth=3, random_state=random_state)
        dt.fit(X_tr, y_tr)
        proba1 = dt.predict_proba(X_te)[:, 1]
        rows.append(("DecisionTree_depth3", *minority_metrics_from_proba(y_te, proba1)))

        # GaussianNB
        gnb = GaussianNB()
        gnb.fit(X_tr, y_tr)
        proba1 = gnb.predict_proba(X_te)[:, 1]
        rows.append(("GaussianNB", *minority_metrics_from_proba(y_te, proba1)))

        # Simple Under-Sampling + RF
        X_res, y_res = RandomUnderSampler(random_state=random_state).fit_resample(X_tr, y_tr)
        rf_under = RandomForestClassifier(n_estimators=100, random_state=random_state)
        rf_under.fit(X_res, y_res)
        proba1 = rf_under.predict_proba(X_te)[:, 1]
        rows.append(("RF_RandomUnderSampler", *minority_metrics_from_proba(y_te, proba1)))

        # Simple Over-Sampling + RF
        X_res, y_res = RandomOverSampler(random_state=random_state).fit_resample(X_tr, y_tr)
        rf_over = RandomForestClassifier(n_estimators=100, random_state=random_state)
        rf_over.fit(X_res, y_res)
        proba1 = rf_over.predict_proba(X_te)[:, 1]
        rows.append(("RF_RandomOverSampler", *minority_metrics_from_proba(y_te, proba1)))

        # RF + class_weight
        rf_cw = RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=random_state)
        rf_cw.fit(X_tr, y_tr)
        proba1 = rf_cw.predict_proba(X_te)[:, 1]
        rows.append(("RF_class_weight", *minority_metrics_from_proba(y_te, proba1)))

        # RF + SMOTE
        X_res, y_res = SMOTE(random_state=random_state).fit_resample(X_tr, y_tr)
        rf_smote = RandomForestClassifier(n_estimators=100, random_state=random_state)
        rf_smote.fit(X_res, y_res)
        proba1 = rf_smote.predict_proba(X_te)[:, 1]
        rows.append(("RF_SMOTE", *minority_metrics_from_proba(y_te, proba1)))

        # RF + ADASYN
        X_res, y_res = ADASYN(random_state=random_state).fit_resample(X_tr, y_tr)
        rf_ada = RandomForestClassifier(n_estimators=100, random_state=random_state)
        rf_ada.fit(X_res, y_res)
        proba1 = rf_ada.predict_proba(X_te)[:, 1]
        rows.append(("RF_ADASYN", *minority_metrics_from_proba(y_te, proba1)))

        # Default XGB (no cost-sensitivity), positive = minority (label flipped)
        y_tr_pos = (y_tr == 0).astype(int)
        y_te_pos = (y_te == 0).astype(int)
        xgb_def = XGBClassifier(
            n_estimators=200, max_depth=4, learning_rate=0.1,
            subsample=0.9, colsample_bytree=0.8,
            scale_pos_weight=1.0, random_state=random_state,
            tree_method="hist", eval_metric="logloss"
        )
        xgb_def.fit(X_tr, y_tr_pos)
        proba_pos = xgb_def.predict_proba(X_te)[:, 1]  # Minority probability
        aucpr_min = average_precision_score(y_te_pos, proba_pos)
        y_pred_pos = (proba_pos >= 0.5).astype(int)
        f1_min = f1_score(y_te_pos, y_pred_pos, pos_label=1)
        rows.append(("XGB_default_noCost", f1_min, aucpr_min))

        # Cost-sensitive XGB
        neg = (y_tr_pos == 0).sum()
        pos = (y_tr_pos == 1).sum()
        spw = neg / max(pos, 1)
        xgb_cs = XGBClassifier(
            n_estimators=200, max_depth=4, learning_rate=0.1,
            subsample=0.9, colsample_bytree=0.8,
            scale_pos_weight=spw, random_state=random_state,
            tree_method="hist", eval_metric="logloss"
        )
        xgb_cs.fit(X_tr, y_tr_pos)
        proba_pos = xgb_cs.predict_proba(X_te)[:, 1]
        aucpr_min = average_precision_score(y_te_pos, proba_pos)
        y_pred_pos = (proba_pos >= 0.5).astype(int)
        f1_min = f1_score(y_te_pos, y_pred_pos, pos_label=1)
        rows.append(("XGB_cost_sensitive", f1_min, aucpr_min))

    df = pd.DataFrame(rows, columns=["Method", "F1_minority", "AUC_PR_minority"])
    summary = df.groupby("Method").agg(["mean", "std"])
    return summary


# -----------------------------
# Run: Shared CV for all comparisons
# -----------------------------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# --- Best parameters from previous results ---
# 1) Best-by-F1:
bestF1_params = dict(
    k=5, n_ants=50, n_iter=30, rho=0.3,
    alpha_edge=2.0, beta_phero=0.5, tau=3.0, max_steps=15,   # 3*k
    alpha=1.0, beta=2.0, gamma=1.0, delta=0.5, use_weights=True
)

# 2) Best-by-AUC-PR:
bestAUC_params = dict(
    k=5, n_ants=50, n_iter=20, rho=0.3,
    alpha_edge=0.5, beta_phero=1.0, tau=3.0, max_steps=15,   # 3*k
    alpha=1.0, beta=1.0, gamma=1.0, delta=0.0, use_weights=True  # Mild weighting
)

# --- Evaluate our method with best params ---
my_bestF1 = eval_my_method(X, y, cv, **bestF1_params, random_state=42)
my_bestAUC = eval_my_method(X, y, cv, **bestAUC_params, random_state=42)

# --- Baselines ---
baseline_summary = eval_baselines(X, y, cv, random_state=42)

# -----------------------------
# Final summary side by side
# -----------------------------
rows = [
    ("MyMethod_bestF1",
     my_bestF1["F1_minority_mean"],  my_bestF1["F1_minority_std"],
     my_bestF1["AUC_PR_minority_mean"], my_bestF1["AUC_PR_minority_std"]),
    ("MyMethod_bestAUC",
     my_bestAUC["F1_minority_mean"],  my_bestAUC["F1_minority_std"],
     my_bestAUC["AUC_PR_minority_mean"], my_bestAUC["AUC_PR_minority_std"]),
]

my_df = pd.DataFrame(rows, columns=["Method","F1_mean","F1_std","AUC_PR_mean","AUC_PR_std"])

# baseline_summary has multi-level columns → flatten column names
base_df = baseline_summary.copy()
base_df.columns = ['_'.join(col) for col in base_df.columns]
base_df = base_df.reset_index().rename(columns={
    "F1_minority_mean":"F1_mean", "F1_minority_std":"F1_std",
    "AUC_PR_minority_mean":"AUC_PR_mean", "AUC_PR_minority_std":"AUC_PR_std"
})

final_df = pd.concat([my_df, base_df[["Method","F1_mean","F1_std","AUC_PR_mean","AUC_PR_std"]]],
                     ignore_index=True)

# Display two tables: sorted by F1 and by AUC-PR
print("\n=== Final Comparison (5-fold CV) — Sorted by F1_minority_mean ===")
print(final_df.sort_values("F1_mean", ascending=False).reset_index(drop=True))

print("\n=== Final Comparison (5-fold CV) — Sorted by AUC_PR_minority_mean ===")
print(final_df.sort_values("AUC_PR_mean", ascending=False).reset_index(drop=True))



=== Final Comparison (5-fold CV) — Sorted by F1_minority_mean ===
                   Method   F1_mean    F1_std  AUC_PR_mean  AUC_PR_std
0             LogReg_noCW  0.963341  0.026824     0.994299    0.006417
1               RF_ADASYN  0.954274  0.021915     0.990752    0.006211
2      XGB_cost_sensitive  0.952720  0.009213     0.992082    0.005532
3         RF_class_weight  0.952283  0.017532     0.989176    0.007341
4      XGB_default_noCost  0.949963  0.015790     0.992246    0.006285
5          KNN_k5_uniform  0.948669  0.028529     0.979702    0.018188
6    RF_RandomOverSampler  0.948255  0.016989     0.989122    0.006902
7                RF_SMOTE  0.946483  0.016655     0.988146    0.006860
8        MyMethod_bestAUC  0.943912  0.020439     0.991301    0.005898
9   RF_RandomUnderSampler  0.940006  0.025400     0.987397    0.008432
10        MyMethod_bestF1  0.939245  0.014950     0.988174    0.006879
11               RF_plain  0.938063  0.020793     0.986920    0.008406
12        